In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 7.9 MB/s eta 0:00:00


In [ ]:
id2label = {
    0: "O",
    1: "B-position",
    2: "I-position",
    3: "B-name",
    4: "I-name",
    5: "B-movie",
    6: "I-movie",
    7: "B-organization",
    8: "I-organization",
    9: "B-company",
    10: "I-company",
    11: "B-book",
    12: "I-book",
    13: "B-address",
    14: "I-address",
    15: "B-scene",
    16: "I-scene",
    17: "B-mobile",
    18: "I-mobile",
    19: "B-email",
    20: "I-email",
    21: "B-game",
    22: "I-game",
    23: "B-government",
    24: "I-government",
    25: "B-QQ",
    26: "I-QQ",
    27: "B-vx",
    28: "I-vx"
  }

In [ ]:
label2id = {
    "B-QQ": 25,
    "B-address": 13,
    "B-book": 11,
    "B-company": 9,
    "B-email": 19,
    "B-game": 21,
    "B-government": 23,
    "B-mobile": 17,
    "B-movie": 5,
    "B-name": 3,
    "B-organization": 7,
    "B-position": 1,
    "B-scene": 15,
    "B-vx": 27,
    "I-QQ": 26,
    "I-address": 14,
    "I-book": 12,
    "I-company": 10,
    "I-email": 20,
    "I-game": 22,
    "I-government": 24,
    "I-mobile": 18,
    "I-movie": 6,
    "I-name": 4,
    "I-organization": 8,
    "I-position": 2,
    "I-scene": 16,
    "I-vx": 28,
    "O": 0
  }

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from peft import PeftConfig, PeftModel
from transformers import pipeline
import numpy as np

tokenizer_kwargs = {'padding':True,'truncation':True,'max_length':512,'return_tensors':'pt'}
tokenizer1 = AutoTokenizer.from_pretrained("gyr66/RoBERTa-ext-lora-chinese-finetuned-ner", trust_remote_code=True)
config1 = PeftConfig.from_pretrained("gyr66/RoBERTa-ext-lora-chinese-finetuned-ner")
model1 = AutoModelForTokenClassification.from_pretrained(
    config1.base_model_name_or_path,
    ignore_mismatched_sizes=True,
    id2label=id2label,
    label2id=label2id
)
inference_model1 = PeftModel.from_pretrained(model1, "gyr66/RoBERTa-ext-lora-chinese-finetuned-ner")


tokenizer2 = AutoTokenizer.from_pretrained("gyr66/RoBERTa-ext-large-lora-chinese-finetuned-ner", trust_remote_code=True)
config2 = PeftConfig.from_pretrained("gyr66/RoBERTa-ext-large-lora-chinese-finetuned-ner")
model2 = AutoModelForTokenClassification.from_pretrained(
    config2.base_model_name_or_path,
    ignore_mismatched_sizes=True,
    id2label=id2label,
    label2id=label2id
)
inference_model2 = PeftModel.from_pretrained(model2, "gyr66/RoBERTa-ext-large-lora-chinese-finetuned-ner")


tokenizer3 = AutoTokenizer.from_pretrained("gyr66/RoBERTa-ext-large-lora-updated-chinese-finetuned-ner", trust_remote_code=True)
config3 = PeftConfig.from_pretrained("gyr66/RoBERTa-ext-large-lora-updated-chinese-finetuned-ner")
model3 = AutoModelForTokenClassification.from_pretrained(
    config3.base_model_name_or_path,
    ignore_mismatched_sizes=True,
    id2label=id2label,
    label2id=label2id
)
inference_model3 = PeftModel.from_pretrained(model3, "gyr66/RoBERTa-ext-large-lora-updated-chinese-finetuned-ner")


nlp_array = [
    pipeline("token-classification",model="gyr66/RoBERTa-ext-large-crf-chinese-finetuned-ner"),
    pipeline("token-classification",model="gyr66/RoBERTa-ext-large-chinese-finetuned-ner"),
    pipeline("token-classification",model="gyr66/bert-base-chinese-finetuned-ner"),
    pipeline("token-classification",model=inference_model1,tokenizer=tokenizer1,trust_remote_code=True),
    pipeline("token-classification",model=inference_model2,tokenizer=tokenizer2,trust_remote_code=True),
    pipeline("token-classification",model=inference_model3,tokenizer=tokenizer3,trust_remote_code=True),
]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at hfl/chinese-roberta-wwm-ext and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForTokenClassification were not initialized from the model checkpoint at hfl/chinese-roberta-wwm-ext-large and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The model 'PeftModelForTokenClassification' is not supported for token-classification. Supported models are ['AlbertForTokenClassification', 'BertForTokenClassification', 'BigBirdForTokenClassification', 'BioGptForTokenClassification', 'BloomForTokenClassification', 'BrosForTokenClassification', 'CamembertForTokenClassification', 'CanineForTokenClassification', 'ConvBertForTokenClassificatio

In [ ]:
import pandas as pd
import os
import re
from collections import Counter


def process_file(filename):
    with open(os.path.join('/content/drive/MyDrive/test_last2', filename), 'r') as f:
        text = f.read()
    result_array = []
    for i in range(6):
      if len(text) > 512:
        truncated_text = text[:512]
      else:
        truncated_text = text
      result_array.append(nlp_array[i](truncated_text))
    aggregated_results = count_entity_values0(result_array)
    final_results = transform_output(aggregated_results, filename.split('.')[0])
    return final_results


res = []
for filename in sorted(os.listdir('/content/drive/MyDrive/test_last2'), key=lambda x: int(x.split('.')[0])):
    file_results = process_file(filename)
    res.extend(file_results)
    # break

df = pd.DataFrame(res)
df.to_csv('/content/sample_data/test1/predict.csv', encoding='utf-8', index=False)

In [ ]:
def transform_output(entities, _id):
    result = []
    # print("flag:",type)
    for entity in entities:
        category = entity['entity']
        word = entity['word'].replace(' ', '')
        start = entity['start']
        end = entity['end'] - 1
        result.append({
            'ID': _id,
            'Category': category,
            'Pos_b': start,
            'Pos_e': end,
            'Privacy': word
        })
    return result

In [ ]:
from collections import Counter
import numpy as np

def count_entity_values0(list_of_lists):
    result = []
    max_length = max(len(lst) for lst in list_of_lists)
    counters = [Counter() for _ in range(max_length)]
    for lst in list_of_lists:
        for i, dictionary in enumerate(lst):
            counters[i][dictionary['entity']] += 1

    # 创建最终的列表
    max_len_index = np.argmax([len(lst) for lst in list_of_lists])

    for i, counter in enumerate(counters):
        entity_most_common = counter.most_common(1)[0][0]

        # 使用 'entity' 值最多的字典初始化一个新字典，并使用源列表中的最长列表的 'index', 'word', 'start' and 'end'
        new_dict = {
            'entity': entity_most_common,
            'index': list_of_lists[max_len_index][i]['index'],
            'word': list_of_lists[max_len_index][i]['word'],
            'start': list_of_lists[max_len_index][i]['start'],
            'end': list_of_lists[max_len_index][i]['end'],
        }
        result.append(new_dict)


    # 初始化一个空列表用来保存聚合结果
    aggregation_results = []

    # 对预测结果进行遍历
    for prediction in result:
        entity = prediction['entity'][2:] # 去掉前面的标识位置的信息
        word = prediction['word']
        start = prediction['start']
        end = prediction['end']

        if len(aggregation_results) == 0 or entity != aggregation_results[-1]['entity'] or (prediction['entity'][0] == 'B' and aggregation_results[-1]['end'] != start - 1):
            # 就创建一个新的词汇并将当前预测添加进去
            aggregation_results.append({
                'entity': entity,
                'word': word,
                'start': start,
                'end': end
            })
        else:
            # 反之，如果实体类型没改变，就将当前预测追加到上一个词汇中
            aggregation_results[-1]['word'] += ' ' + word
            aggregation_results[-1]['end'] = end
    return aggregation_results